<a href="https://colab.research.google.com/github/atakhadivi/Algorithmic-Trading/blob/master/ma200bbrsi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install everything we need to import
!pip3 install -q python-binance
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib
!pip3 install -q tinydb

     |████████████████████████████████| 358kB 9.0MB/s 
     |████████████████████████████████| 61kB 5.1MB/s 
     |████████████████████████████████| 1.1MB 16.4MB/s 
     |████████████████████████████████| 2.7MB 36.8MB/s 
     |████████████████████████████████| 3.1MB 33.7MB/s 
     |████████████████████████████████| 61kB 4.2MB/s 
     |████████████████████████████████| 235kB 38.7MB/s 
Selecting previously unselected package libta-lib0.
(Reading database ... 144467 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is 

In [ ]:
# import every thing need to bot work
import datetime
from binance.client import Client
import pandas as pd
from talib import RSI, MA, BBANDS
import numpy as np
from tinydb import TinyDB, Query
import time
import requests

In [ ]:
# aal symbols for tradiong
symbol = [
  "BTCUSDT",
  "ETHUSDT",
  "LTCUSDT",
  "BNBUSDT",
  "NEOUSDT",
  "QTUMUSDT",
  "ADAUSDT",
  "XRPUSDT",
  "EOSUSDT",
  "IOTAUSDT",
  "XLMUSDT",
  "ONTUSDT",
  "TRXUSDT",
  "ETCUSDT",
  "ICXUSDT",
  "NULSUSDT",
  "VETUSDT",
  "LINKUSDT",
  "WAVESUSDT",
  "BTTUSDT",
  "ONGUSDT",
  "HOTUSDT",
  "ZILUSDT",
  "ZRXUSDT",
  "FETUSDT",
  "BATUSDT",
  "XMRUSDT",
  "ZECUSDT",
  "IOSTUSDT",
  "CELRUSDT",
  "DASHUSDT",
  "NANOUSDT",
  "OMGUSDT",
  "THETAUSDT",
  "ENJUSDT",
  "MITHUSDT",
  "MATICUSDT",
  "ATOMUSDT",
  "TFUELUSDT",
  "ONEUSDT",
  "FTMUSDT",
  "ALGOUSDT",
  "GTOUSDT",
  "ERDUSDT",
  "DOGEUSDT",
  "DUSKUSDT",
  "ANKRUSDT",
  "WINUSDT",
  "COSUSDT",
  "NPXSUSDT",
  "COCOSUSDT",
  "MTLUSDT",
  "TOMOUSDT",
  "PERLUSDT",
  "DENTUSDT",
  "MFTUSDT",
  "KEYUSDT",
  "STORMUSDT",
  "DOCKUSDT",
  "WANUSDT",
  "FUNUSDT",
  "CVCUSDT",
  "CHZUSDT",
  "BANDUSDT",
  "BEAMUSDT",
  "XTZUSDT",
  "RENUSDT",
  "RVNUSDT",
  "HCUSDT",
  "HBARUSDT",
  "NKNUSDT",
  "STXUSDT",
  "KAVAUSDT",
  "ARPAUSDT",
  "IOTXUSDT",
  "RLCUSDT",
  "MCOUSDT",
  "CTXCUSDT",
  "BCHUSDT",
  "TROYUSDT",
  "VITEUSDT",
  "FTTUSDT",
  "OGNUSDT",
  "DREPUSDT",
  "TCTUSDT",
  "WRXUSDT",
  "BTSUSDT",
  "LSKUSDT",
  "BNTUSDT",
  "LTOUSDT",
  "STRATUSDT",
  "AIONUSDT",
  "MBLUSDT",
  "COTIUSDT",
  "STPTUSDT",
  "WTCUSDT",
  "DATAUSDT",
  "XZCUSDT",
]

In [ ]:
# telegram bot to send signals
def telegram_bot_sendtext(bot_message):

    bot_token = ''
    bot_chatID = ''
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message

    response = requests.get(send_text)

    return response.json()

In [ ]:
# api key and api secret
api_key = ''
api_secret = ''

In [ ]:
# database creation
db = TinyDB('db.json')
user = Query()
for i in symbol:
  db.insert({'symbol':i,'rsi':100})

In [ ]:
# create data and bot
def get_data(symbol):
  # create pandas dataframe from binance
  client = Client(api_key, api_secret)
  df = client.get_historical_klines(symbol, Client.KLINE_INTERVAL_5MINUTE, "1 day ago UTC")
  df = pd.DataFrame(df,columns= ['datetime', 'open', 'high', 'low', 'close', 'volume', 'Close_time','qoute assest volume','number of trades','taker buy base assest volume','taker buy qoute assest volume','ignore'])
  df['datetime'] = pd.to_datetime(df['datetime'],unit = 'ms')
  df.set_index('datetime',inplace=True)
  df.drop(['Close_time','qoute assest volume','number of trades','taker buy base assest volume','taker buy qoute assest volume','ignore'],axis=1,inplace=True)
  # candles.drop(candles.tail(1).index,inplace=True)
  # candles.drop(candles.head(1).index,inplace=True)
  # return df
  # candles.to_csv('out.csv')
  df = pd.DataFrame(data=df['close'], dtype=np.float64)
  # float_data = [float(x) for x in df['close']]
  close = df['close'].values
  # calculate rsi and moving average
  up, mid, low = BBANDS(close, timeperiod=21, nbdevup=2, nbdevdn=2, matype=0)
  ma = MA(close, timeperiod=200, matype=0)
  rsi = RSI(close, timeperiod=14)
  bb_length = len(low)
  rsi_length = len(rsi)
  ma_length = len(ma)
  sell_price = df['close'].tail(1).values + ((df['close'].tail(1).values / 100.00)*1.20)
  # if rsi[rsi_length - 1] < 70.00 and :
  #   return (rsi[rsi_length - 1])
  # calculate and condition for sell and buy
  if df['close'].tail(1).values > ma[ma_length - 1]:
    if rsi[rsi_length - 1] > 30.00:
      if db.get(user.symbol== symbol)['rsi'] < 30.00:
        if db.get(user.symbol == symbol)['close'] < db.get(user.symbol == symbol)['bb']:
          text = "{} \n RSI: {} \n MA 200: {} \n price: {} \n Sell Price: {}".format(symbol,rsi[rsi_length - 1],ma[ma_length - 1],df['close'].tail(1).values,sell_price)
          telegram_bot_sendtext(text)
  # db.insert({'symbol':symbol,'rsi':rsi[rsi_length - 1],'bb':low[bb_length - 1]})
  # update database for new data
  close_update = df['close'].tail(1).to_list()
  db.update({'rsi':rsi[rsi_length - 1],'MA 200':ma[ma_length - 1],'close':close_update[0]}, user.symbol == symbol)
  # if df['close'].tail(1) < low[bb_length - 1] and rsi[rsi_length - 1] < 30:
  #   return(rsi.tail(1))

In [ ]:
try:
  while True:
      for i in symbol:
        # print (get_data(i))
        get = get_data(i)
        if  get is not None:
          print (get)
  # sleep(250)
except ValueError as ve:
  print(ve)

KeyError: ignored